## Deliverable 2. Create a Customer Travel Destinations Map.

In [12]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [13]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kruisfontein,ZA,-34.0033,24.7314,62.89,76,69,5.32,broken clouds
1,1,Shelburne,CA,44.0787,-80.2041,14.90,86,90,16.64,overcast clouds
2,2,Bluff,NZ,-46.6000,168.3333,59.47,76,18,3.33,few clouds
3,3,Rocha,UY,-34.4833,-54.3333,61.77,55,0,4.83,clear sky
4,4,Torbay,CA,47.6666,-52.7314,40.28,90,100,3.00,light rain


In [14]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [15]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(15)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Vaini,TO,-21.2000,-175.2000,80.40,65,46,8.66,scattered clouds
6,6,Cockburn Town,TC,21.4612,-71.1419,76.59,70,81,17.96,broken clouds
7,7,Hilo,US,19.7297,-155.0900,80.33,62,20,9.22,few clouds
10,10,Rosario Oeste,BR,-14.8361,-56.4275,76.28,89,81,1.50,moderate rain
12,12,Rikitea,PF,-23.1203,-134.9692,79.18,65,27,8.81,scattered clouds
17,17,Salalah,OM,17.0151,54.0924,75.29,50,0,4.61,clear sky
18,18,San Pedro,PH,14.3500,121.0167,85.32,69,94,8.57,overcast clouds
19,19,Honiara,SB,-9.4333,159.9500,84.58,73,91,2.04,overcast clouds
24,24,Auki,SB,-8.7676,160.7034,81.93,80,100,5.35,light rain
25,25,Atuona,PF,-9.8000,-139.0333,78.82,82,67,13.15,light rain


In [16]:
# 4a. Determine if there are any empty rows.

preferred_cities_df.count()

City_ID                221
City                   221
Country                220
Lat                    221
Lng                    221
Max Temp               221
Humidity               221
Cloudiness             221
Wind Speed             221
Current Description    221
dtype: int64

In [17]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_cities_df = preferred_cities_df.dropna()

In [18]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Vaini,TO,80.40,scattered clouds,-21.2000,-175.2000,
6,Cockburn Town,TC,76.59,broken clouds,21.4612,-71.1419,
7,Hilo,US,80.33,few clouds,19.7297,-155.0900,
10,Rosario Oeste,BR,76.28,moderate rain,-14.8361,-56.4275,
12,Rikitea,PF,79.18,scattered clouds,-23.1203,-134.9692,
...,...,...,...,...,...,...,...
702,Tawau,MY,78.93,overcast clouds,4.2498,117.8871,
705,Gangakher,IN,82.35,few clouds,18.9500,76.7500,
707,Bara,NG,77.32,few clouds,10.3744,10.7288,
708,Barra Patuca,HN,76.35,overcast clouds,15.8000,-84.2833,


In [19]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f.# Grab the first hotel from the results and store the name.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.") 
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [21]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.replace("", np.nan, inplace = True)
hotel_df.dropna().head()
    

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Vaini,TO,80.40,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
6,Cockburn Town,TC,76.59,broken clouds,21.4612,-71.1419,Osprey Beach Hotel
7,Hilo,US,80.33,few clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
10,Rosario Oeste,BR,76.28,moderate rain,-14.8361,-56.4275,Alvorada Palace accomodation
12,Rikitea,PF,79.18,scattered clouds,-23.1203,-134.9692,People ThankYou


In [22]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """""
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]


In [24]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             #dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
#Take a screenshot of your map and save it to the Vacation_Search folder as WeatherPy_vacation_map.png